In [ ]:
!pip install -qU langchain-huggingface

In [ ]:
!pip install neo4j-driver

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
import pandas as pd
from neo4j import GraphDatabase, basic_auth, Result
from tqdm import tqdm
from google.colab import userdata

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
driver = GraphDatabase.driver(
  userdata.get("neo4j_url"),
  auth=basic_auth(userdata.get("neo4j_username"), userdata.get("neo4j_password")))

In [ ]:
tweets = driver.execute_query("""
MATCH (t:Tweet) WHERE t.text IS NOT NULL
RETURN t.id AS id, t.text AS text
""",result_transformer_=Result.to_df)

In [ ]:
tweets

,id,text
0,1371815021265747970,Proud to help @educationgovuk deliver critical...
1,1371778287970705408,[Webinar] Produktdatenmanagement mit der Graph...
2,1371687808847519748,[Webinar] Graph Data Science with #Neo4j – Ita...
3,1371787098945114113,RT @rotnroll666: Neo4j Cypher-DSL 2021.1 relea...
4,1371673079517171712,Discover how financial institutions can use gr...
...,...,...
2141,1335606350739595264,RT @StackDevJobs: Software Developer Kubernete...
2142,1335553487959187458,RT @rotnroll666: Music 2020. Wrapped. Without ...
2143,1335540744535281666,Making sense of 2020's mad calendar with Neo4j...
2144,1335486890548617216,#NODES2020 Extended tomorrow 7am UTC/8am CET/1...


In [ ]:
embeddings_list = []
for _, tweet in tqdm(tweets.iterrows(), total=len(tweets)):
    embedding = embeddings.embed_query(tweet['text'])
    embeddings_list.append({'id': tweet['id'], 'embedding': embedding})

100%|██████████| 2146/2146 [00:30<00:00, 70.74it/s]


In [ ]:
pd.DataFrame(embeddings_list).to_csv('out.csv', index=False)